In [2]:
import scipy.optimize as so
import numpy as np
import h5py as h5
    

In [3]:
def UFid(U,V,Dim):
#    return np.sum(abs(U-V)**2)
    return abs(np.trace(np.matmul(U.conj().T,V))/Dim)**2

In [4]:
def QuantumFourier(Dim): 
    pi=np.arccos(-1.)
    x=np.array(range(Dim))
    return np.exp( 1j*2*pi*np.reshape( np.kron(x,x), [Dim,Dim] )/Dim )/np.sqrt(Dim)

In [5]:
def RandomUnitary(Dim):
    return np.linalg.qr( np.random.randn(Dim,Dim)+1j*np.random.randn(Dim,Dim) )[0]

In [6]:
def UFunction(Phase,N,F,Dim):
    U=np.eye(Dim)
    for k in range(N):
        U= np.exp( 1j*np.append([0] ,Phase[k*(Dim-1):(k+1)*(Dim-1)] ) )[:,np.newaxis]*U
        U=np.matmul(F,U)
    return np.exp(1j*np.append( [0], Phase[N*(Dim-1):(N+1)*Dim-1]))[:,np.newaxis]*U

In [7]:
def PUMIOpt(F,Dim,N,U):
    Seed = np.ones( (N+1)*(Dim-1) )
    TargetOpt = lambda x: 1-UFid( U , UFunction( x , N , F , Dim ) , Dim )
    argsmin = {"method": "L-BFGS-B"}
    Res=so.basinhopping(
        TargetOpt,
        Seed,
        minimizer_kwargs=argsmin,
        niter=5*10**2,
        disp=False,
        )
    return Res.fun

In [ ]:
import pandas as pd
from tqdm import tqdm

worst_number = 100

infids_julia = []

infids_python = []

for Dim in range(3,4):

    L   = 0
    data_file = "data/HaarDistributed_Dim="+str(Dim)+"_Lay=Dim+"+str(L)+".h5"

    F = QuantumFourier(Dim)

    with h5.File( data_file, "r" ) as file:

        infids = file['infi'][:]
    
    monte_carlo = infids.shape[0]
    infids_mean = infids.mean(axis=0)

    df = pd.DataFrame(
        data=infids_mean,
        columns=["Infidelities"], 
        )
    

    mask = df > 10**(-10)
    worst_table = df[mask].dropna()

    infids_julia.append( worst_table )

    worst_idx = worst_table.index.to_list()

    with h5.File( "HaarUnitary.h5", "r" ) as file:
        
        UU = file["unitarydim={}".format(Dim)][:,:,:]

    
    infids_BH = np.zeros([len(worst_idx),monte_carlo])
    
    for i,index in tqdm(enumerate(worst_idx)):

        Unitary = UU[ index, :,: ]

        for mc in range(monte_carlo):

            infids_BH[i,mc] = PUMIOpt(
                F = F, #Fourier
                Dim=Dim, #Dimension
                N=Dim + L,#layers number
                U = Unitary, #Target unitary
                )
            
    infids_BH = infids_BH.mean(axis=1)
    worst_table_BH = pd.DataFrame(
        data = infids_BH,
        columns="Infidelities",
        index = worst_idx,
    )

    infids_julia.append(worst_table_BH)


 17%|█▋        | 5/30 [01:02<05:12, 12.51s/it]


KeyboardInterrupt: 

In [58]:

len(worst_idx)

473

In [91]:
np.save( 'julia_vs_python', infids_comparison )

In [47]:
def main():
    Repe=100
    Dim=4
    N=Dim
    
    F = QuantumFourier(Dim)
    file=h5.File("FFdataDim=4.h5","r") # Cambiar Nombre archivo 
    UU=file["re"][:,:,:].T+1j*file["im"][:,:,:].T
    
    ind2=file["Index"][:]-1
    MC=len(ind2)
    file.close()
    Data=np.zeros([MC,Repe])
    l=np.where(Data[:,0]<1)
    print(Dim,"-----",MC)
    for j in range(Repe):
        for i in range(MC):
            Data[i,j]=PUMIOpt( F,Dim,N,UU[:,:,ind2[ l[0][i] ]] )
        l2=np.where(Data[l,j]<10**(-10))
        for h in range(np.size(l2[0])):
            Data[l2[0][h],:]=Data[l2[0][h],j]
        l=np.where(Data[:,j]>10**(-10))
        MC=np.size(l[0])
        print(j,"-----",np.size(l[0]))

    file=h5.File("FFdataDim=4py.h5","w") #Cambiar nombre al archivo de salida
    file["infi"]=Data
    file["index1"]=l[0]
    file["index2"]=l2[0]
    file.close()